In [ ]:
# we can start coding here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb

# Set Pandas display options to show all columns and rows
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/Shareddrives/Humana_mays_Competetion/Data/"

In [ ]:
#target data
target_train = pd.read_csv(data_path+"target_train.csv")
target_holdout = pd.read_csv(data_path+"target_holdout.csv")

#pharamacy claims
rxclms_train = pd.read_csv(data_path+"rxclms_train.csv")
rxclms_holdout = pd.read_csv(data_path+"rxclms_holdout.csv")

#medical claims
medclms_train = pd.read_csv(data_path+"medclms_train.csv")
medclms_holdout = pd.read_csv(data_path+"medclms_holdout.csv")

In [ ]:
print(target_train.shape)
print(target_holdout.shape)


print(rxclms_train.shape)
print(rxclms_holdout.shape)


print(medclms_train.shape)
print(medclms_holdout.shape)

(1232, 10)
(420, 8)
(32133, 24)
(6669, 24)
(100159, 27)
(23232, 27)


#Data Exploration

##Target Train

In [ ]:
print(target_train.columns)

Index(['id', 'therapy_id', 'therapy_start_date', 'therapy_end_date',
       'tgt_ade_dc_ind', 'race_cd', 'est_age', 'sex_cd', 'cms_disabled_ind',
       'cms_low_income_ind'],
      dtype='object')


In [ ]:
target_train.head(5)

,id,therapy_id,therapy_start_date,therapy_end_date,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind
0,1005597228,1005597228-TAGRISSO-1,2020-03-11T00:00:00.000+0000,2020-09-07,0,1.0,NaN,NaN,NaN,NaN
1,1005964858,1005964858-TAGRISSO-1,2021-08-23T00:00:00.000+0000,2021-09-22,1,1.0,86.0,M,0.0,0.0
2,1007548572,1007548572-TAGRISSO-1,2022-01-17T00:00:00.000+0000,2022-07-16,0,1.0,79.0,F,0.0,1.0
3,1009508044,1009508044-TAGRISSO-1,2020-01-23T00:00:00.000+0000,2020-07-21,0,1.0,70.0,F,1.0,1.0
4,1028064791,1028064791-TAGRISSO-1,2021-02-23T00:00:00.000+0000,2021-08-22,0,2.0,74.0,F,1.0,1.0


In [ ]:
target_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1232 non-null   int64  
 1   therapy_id          1232 non-null   object 
 2   therapy_start_date  1232 non-null   object 
 3   therapy_end_date    1232 non-null   object 
 4   tgt_ade_dc_ind      1232 non-null   int64  
 5   race_cd             1164 non-null   float64
 6   est_age             1149 non-null   float64
 7   sex_cd              1149 non-null   object 
 8   cms_disabled_ind    1149 non-null   float64
 9   cms_low_income_ind  1149 non-null   float64
dtypes: float64(4), int64(2), object(4)
memory usage: 96.4+ KB


In [ ]:
target_train.isnull().sum()

id                     0
therapy_id             0
therapy_start_date     0
therapy_end_date       0
tgt_ade_dc_ind         0
race_cd               68
est_age               83
sex_cd                83
cms_disabled_ind      83
cms_low_income_ind    83
dtype: int64

Null values found in columns race_cd, est_age, sex_cd, cms_disabled_ind, cms_low_income_ind

Exploring each column seperately in this data

###id

In [ ]:
#check if there are any duplicate ids
target_train[target_train['id'].duplicated(keep=False)]
#there are no duplicate ids - each member has undergone just one therapy

,id,therapy_id,therapy_start_date,therapy_end_date,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind


###therapy-id

In [ ]:
#split it up into three columns to extract any useful information
target_train[['member_id', 'drug_name', 'therapy_num']] = target_train['therapy_id'].str.split('-', expand=True)
target_train

,id,therapy_id,therapy_start_date,therapy_end_date,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind,member_id,drug_name,therapy_num
0,1005597228,1005597228-TAGRISSO-1,2020-03-11T00:00:00.000+0000,2020-09-07,0,1.0,NaN,NaN,NaN,NaN,1005597228,TAGRISSO,1
1,1005964858,1005964858-TAGRISSO-1,2021-08-23T00:00:00.000+0000,2021-09-22,1,1.0,86.0,M,0.0,0.0,1005964858,TAGRISSO,1
2,1007548572,1007548572-TAGRISSO-1,2022-01-17T00:00:00.000+0000,2022-07-16,0,1.0,79.0,F,0.0,1.0,1007548572,TAGRISSO,1
3,1009508044,1009508044-TAGRISSO-1,2020-01-23T00:00:00.000+0000,2020-07-21,0,1.0,70.0,F,1.0,1.0,1009508044,TAGRISSO,1
4,1028064791,1028064791-TAGRISSO-1,2021-02-23T00:00:00.000+0000,2021-08-22,0,2.0,74.0,F,1.0,1.0,1028064791,TAGRISSO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,1005265852,1005265852-TAGRISSO-1,2022-01-18T00:00:00.000+0000,2022-05-01,1,5.0,91.0,F,0.0,0.0,1005265852,TAGRISSO,1
1228,1017236720,1017236720-TAGRISSO-1,2019-09-10T00:00:00.000+0000,2020-03-08,0,1.0,79.0,M,0.0,0.0,1017236720,TAGRISSO,1
1229,1059781692,1059781692-TAGRISSO-1,2020-03-10T00:00:00.000+0000,2020-09-06,0,4.0,81.0,F,0.0,1.0,1059781692,TAGRISSO,1
1230,1106680931,1106680931-TAGRISSO-1,2021-03-01T00:00:00.000+0000,2021-03-31,0,1.0,50.0,M,1.0,0.0,1106680931,TAGRISSO,1


In [ ]:
target_train['member_id'] = pd.to_numeric(target_train['member_id'], errors='coerce')
(target_train['member_id'] - target_train['id']).describe()

count    1232.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64

In [ ]:
target_train['drug_name'].value_counts()

TAGRISSO    1232
Name: drug_name, dtype: int64

In [ ]:
target_train['therapy_num'].value_counts()

1    1232
Name: therapy_num, dtype: int64

Each member has undergone utmost one therapy with the drug named TAGRISSO.
Hence therapy_id in itself does not hold any information which could be used for predictions. But retaining this column as it is required to join with other tables

### therapy_start_date/therapy_end_date

In [ ]:
target_train['therapy_start_date'].value_counts()

2019-11-04T00:00:00.000+0000    10
2022-01-24T00:00:00.000+0000     9
2021-01-11T00:00:00.000+0000     9
2021-01-14T00:00:00.000+0000     9
2019-10-18T00:00:00.000+0000     7
2022-01-25T00:00:00.000+0000     7
2020-01-07T00:00:00.000+0000     7
2021-05-27T00:00:00.000+0000     6
2022-02-01T00:00:00.000+0000     6
2022-01-06T00:00:00.000+0000     6
2022-01-05T00:00:00.000+0000     6
2022-05-18T00:00:00.000+0000     6
2022-03-22T00:00:00.000+0000     6
2019-10-01T00:00:00.000+0000     6
2020-09-30T00:00:00.000+0000     6
2021-02-01T00:00:00.000+0000     6
2020-02-20T00:00:00.000+0000     6
2021-01-26T00:00:00.000+0000     6
2021-01-04T00:00:00.000+0000     6
2022-04-27T00:00:00.000+0000     5
2019-07-02T00:00:00.000+0000     5
2022-01-27T00:00:00.000+0000     5
2021-01-29T00:00:00.000+0000     5
2022-01-10T00:00:00.000+0000     5
2021-01-07T00:00:00.000+0000     5
2021-01-15T00:00:00.000+0000     5
2022-04-05T00:00:00.000+0000     5
2021-01-05T00:00:00.000+0000     5
2019-09-03T00:00:00.

In [ ]:
target_train['therapy_end_date'].value_counts()

2020-07-05    6
2021-01-16    5
2022-07-16    5
2022-03-13    5
2021-02-13    5
2021-06-09    5
2022-07-23    5
2021-07-25    5
2020-02-02    5
2021-07-31    5
2021-07-13    4
2022-10-16    4
2022-03-25    4
2021-04-11    4
2022-04-06    4
2021-02-10    4
2021-02-14    4
2020-12-12    4
2021-07-07    4
2020-08-23    4
2020-07-15    4
2020-08-09    4
2021-02-12    4
2020-02-09    4
2022-05-18    4
2020-07-06    4
2022-07-11    4
2019-12-29    4
2020-09-13    4
2022-07-04    4
2020-07-01    4
2020-01-18    4
2022-09-18    4
2021-08-31    4
2021-07-10    4
2020-04-13    4
2019-10-31    4
2022-11-14    3
2022-07-24    3
2021-07-03    3
2021-08-25    3
2021-05-08    3
2020-04-18    3
2021-03-29    3
2021-10-31    3
2021-07-28    3
2022-03-26    3
2022-01-30    3
2021-07-12    3
2022-04-23    3
2021-12-29    3
2022-05-21    3
2020-07-10    3
2021-08-29    3
2020-09-06    3
2022-06-17    3
2020-05-20    3
2022-05-04    3
2020-04-26    3
2021-03-20    3
2020-06-17    3
2021-08-07    3
2021-03-

In [ ]:
#converting to date time format for easy manipulation
target_train['therapy_start_date'] = pd.to_datetime(target_train['therapy_start_date'])
target_train['therapy_end_date'] = pd.to_datetime(target_train['therapy_end_date'])

#normalizing end date to UTC format - for calculating therapy duration
target_train['therapy_end_date'] = target_train['therapy_end_date'].dt.tz_localize('UTC')

#calculate number of days from therapy start to therapy end date
target_train['therapy_duration'] = (target_train['therapy_end_date'] - target_train['therapy_start_date']).dt.days

#range of therapy duration
target_train['therapy_duration'].describe()

count    1232.000000
mean      113.648539
std        65.410389
min         7.000000
25%        30.000000
50%       121.000000
75%       180.000000
max       180.000000
Name: therapy_duration, dtype: float64

We have members who have just had 7 days of therapy and also members who have undergone upto 180 days of therapy

### tgt_ade_dc_ind

In [ ]:
target_train['tgt_ade_dc_ind'].value_counts()

0    1115
1     117
Name: tgt_ade_dc_ind, dtype: int64

Highly imbalanced dataset - we only have 117 members who discontinued therapy within 6 months and had an adverse drug affect (ade) - We might need to balance this dataset before modelling

There are 581 rows where tgt_ade_dc_ind = 0 but the therapy duration is under 180 days.

These patients fall into either of the cases:
* Therapies that end prematurely with no reported ADEs
* Therapies where the member changes to another insurance plan or dies before 180 days

###race_cd

In [ ]:
target_train['race_cd'].value_counts()

1.0    699
4.0    152
2.0    145
0.0     84
5.0     42
3.0     37
6.0      5
Name: race_cd, dtype: int64

These might be encoded to numbers. will keep these features for now and remove later as it could lead to low fairness score

###est_age               

In [ ]:
target_train['est_age'].describe()

count    1149.000000
mean       73.771976
std         8.517059
min        38.000000
25%        68.000000
50%        73.000000
75%        80.000000
max        96.000000
Name: est_age, dtype: float64

We have people who fall in the age between 38 years and 96 years - good to know!

###sex_cd                

In [ ]:
target_train['sex_cd'].value_counts()

F    815
M    334
Name: sex_cd, dtype: int64

Lot of females in the dataset than males - keep them in the model for now and remove later

###cms_disabled_ind - indicates if the member is classified as disabled by CMS
(Centers for Medicare & Medicaid Services) - identify individuals who meet the criteria for disability-related benefits or services.

In [ ]:
target_train['cms_disabled_ind'].value_counts()

0.0    980
1.0    169
Name: cms_disabled_ind, dtype: int64

###cms_low_income_ind -  indicates if the member recieves low income subsidies from CMS
Low-income subsidies are intended to help eligible individuals with limited income and resources afford their Medicare Part D prescription drug coverage.

In [ ]:
target_train['cms_low_income_ind'].value_counts()

0.0    703
1.0    446
Name: cms_low_income_ind, dtype: int64

## rxclms  
All pharmacy claims for a individual during the time 90 days before their Osimertinib therapy and through the end of therapy. This data includes service and process dates, drug identifier codes (NDC) and indicators for drug codes of interest

In [ ]:
print(rxclms_train.shape)

(32133, 24)


In [ ]:
print(rxclms_train.columns)

Index(['therapy_id', 'document_key', 'ndc_id', 'service_date', 'process_date',
       'pay_day_supply_cnt', 'rx_cost', 'tot_drug_cost_accum_amt',
       'reversal_ind', 'mail_order_ind', 'generic_ind', 'maint_ind',
       'gpi_drug_group_desc', 'gpi_drug_class_desc', 'hum_drug_class_desc',
       'strength_meas', 'metric_strength', 'specialty_ind', 'clm_type',
       'ddi_ind', 'anticoag_ind', 'diarrhea_treat_ind', 'nausea_treat_ind',
       'seizure_treat_ind'],
      dtype='object')


In [ ]:
rxclms_train.head()

,therapy_id,document_key,ndc_id,service_date,process_date,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,reversal_ind,mail_order_ind,generic_ind,maint_ind,gpi_drug_group_desc,gpi_drug_class_desc,hum_drug_class_desc,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind
0,1009508044-TAGRISSO-1,A184611654291011,169266015,2018-03-02T00:00:00.000Z,2019-12-04T00:00:00.000Z,30,919.78,2830.22,N,N,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.0,NONSPCL,rx,0,0,0,0,0
1,1023838279-TAGRISSO-1,A184877988141011,69097022416,2018-03-28T00:00:00.000Z,2020-03-13T00:00:00.000Z,28,5.00,1138.66,N,N,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.0,NONSPCL,rx,0,0,0,0,0
2,1023838279-TAGRISSO-1,A185952462961011,60505257908,2018-07-14T00:00:00.000Z,2020-03-13T00:00:00.000Z,30,7.30,2693.11,N,N,GENERIC,MAINT,*ANTIHYPERLIPIDEMICS*,*HMG COA REDUCTASE INHIBITORS**,CHOLESTEROL,MG,20.0,NONSPCL,rx,0,0,0,0,0
3,1023838279-TAGRISSO-1,A186219930881011,65862057290,2018-08-09T00:00:00.000Z,2020-03-17T00:00:00.000Z,30,12.20,2718.31,N,N,GENERIC,MAINT,*ANTIHYPERTENSIVES*,*ANGIOTENSIN II RECEPTOR ANTAGONISTS**,CARDIOLOGY - HYPERTENSION AGENTS,MG,160.0,NONSPCL,rx,0,0,0,0,0
4,1071647492-TAGRISSO-1,A194013465121011,16252060144,2019-01-01T00:00:00.000Z,2020-03-03T00:00:00.000Z,90,9.90,0.00,N,Y,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.0,NONSPCL,rx,0,0,0,0,0


In [ ]:
rxclms_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   therapy_id               32133 non-null  object 
 1   document_key             32133 non-null  object 
 2   ndc_id                   32133 non-null  int64  
 3   service_date             32133 non-null  object 
 4   process_date             32133 non-null  object 
 5   pay_day_supply_cnt       32133 non-null  int64  
 6   rx_cost                  32133 non-null  float64
 7   tot_drug_cost_accum_amt  32133 non-null  float64
 8   reversal_ind             32133 non-null  object 
 9   mail_order_ind           32133 non-null  object 
 10  generic_ind              32133 non-null  object 
 11  maint_ind                32133 non-null  object 
 12  gpi_drug_group_desc      30234 non-null  object 
 13  gpi_drug_class_desc      30234 non-null  object 
 14  hum_drug_class_desc   

In [ ]:
rxclms_train.isnull().sum()

therapy_id                    0
document_key                  0
ndc_id                        0
service_date                  0
process_date                  0
pay_day_supply_cnt            0
rx_cost                       0
tot_drug_cost_accum_amt       0
reversal_ind                  0
mail_order_ind                0
generic_ind                   0
maint_ind                     0
gpi_drug_group_desc        1899
gpi_drug_class_desc        1899
hum_drug_class_desc        1899
strength_meas              2148
metric_strength            2148
specialty_ind                 0
clm_type                      0
ddi_ind                       0
anticoag_ind                  0
diarrhea_treat_ind            0
nausea_treat_ind              0
seizure_treat_ind             0
dtype: int64

Missing values found in columns - gpi_drug_group_desc, gpi_drug_class_desc, hum_drug_class_desc

###therapy_id

In [ ]:
rxclms_train['hum_drug_class_desc'].value_counts()

CHEMOTHERAPY                                5078
CARDIOLOGY - HYPERTENSION AGENTS            4016
PAIN MGMT - NARCOTIC ANALGESICS             1708
ANTI-INFECTIVES                             1556
CHOLESTEROL                                 1404
GASTROINTESTINAL DISEASE                    1222
GASTROINTESTINAL DISEASE - GERD             1221
MENTAL HEALTH - DEPRESSION AGENTS           1194
DIABETES                                     972
STEROIDS                                     964
ANTICONVULSANTS                              928
CARDIOLOGY - BLOOD THINNERS                  855
MENTAL HEALTH - ANXIETY AGENTS               748
RESPIRATORY - ASTHMA                         699
DERMATOLOGY                                  698
RESPIRATORY - COPD                           604
NUTRITIONAL/ VITAMIN                         563
COUGH/ COLD/ ALLERGY                         561
THYROID                                      560
GENITOURINARY                                455
PAIN MGMT - NSAID AG

In [ ]:
#check if there are multiple entries for same therapy id
rxclms_train[rxclms_train.duplicated(['therapy_id'], keep = False)]
#there are multiple claims reported by members
# need to create aggregated features from the data before joining with the target

,therapy_id,document_key,ndc_id,service_date,process_date,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,reversal_ind,mail_order_ind,generic_ind,maint_ind,gpi_drug_group_desc,gpi_drug_class_desc,hum_drug_class_desc,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind
0,1009508044-TAGRISSO-1,A184611654291011,169266015,2018-03-02T00:00:00.000Z,2019-12-04T00:00:00.000Z,30,919.78,2830.22,N,N,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.000,NONSPCL,rx,0,0,0,0,0
1,1023838279-TAGRISSO-1,A184877988141011,69097022416,2018-03-28T00:00:00.000Z,2020-03-13T00:00:00.000Z,28,5.00,1138.66,N,N,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.000,NONSPCL,rx,0,0,0,0,0
2,1023838279-TAGRISSO-1,A185952462961011,60505257908,2018-07-14T00:00:00.000Z,2020-03-13T00:00:00.000Z,30,7.30,2693.11,N,N,GENERIC,MAINT,*ANTIHYPERLIPIDEMICS*,*HMG COA REDUCTASE INHIBITORS**,CHOLESTEROL,MG,20.000,NONSPCL,rx,0,0,0,0,0
3,1023838279-TAGRISSO-1,A186219930881011,65862057290,2018-08-09T00:00:00.000Z,2020-03-17T00:00:00.000Z,30,12.20,2718.31,N,N,GENERIC,MAINT,*ANTIHYPERTENSIVES*,*ANGIOTENSIN II RECEPTOR ANTAGONISTS**,CARDIOLOGY - HYPERTENSION AGENTS,MG,160.000,NONSPCL,rx,0,0,0,0,0
4,1071647492-TAGRISSO-1,A194013465121011,16252060144,2019-01-01T00:00:00.000Z,2020-03-03T00:00:00.000Z,90,9.90,0.00,N,Y,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.000,NONSPCL,rx,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,1013396924-TAGRISSO-1,M507381525551001,43547027509,2020-12-03T00:00:00.000Z,2020-12-04T00:00:00.000Z,30,5.20,22.60,N,N,GENERIC,MAINT,*PSYCHOTHERAPEUTIC AND NEUROLOGICAL AGENTS - M...,*ANTIDEMENTIA AGENTS**,MENTAL HEALTH - ANTI-DEMENTIA AGENTS,MG,5.000,NONSPCL,rx,0,0,0,0,0
32129,1050705030-TAGRISSO-1,M517443068761011,61314054701,2021-12-10T00:00:00.000Z,2021-12-28T00:00:00.000Z,90,24.59,14676.67,N,Y,GENERIC,MAINT,*OPHTHALMIC AGENTS*,*PROSTAGLANDINS - OPHTHALMIC**,OPHTHALMOLOGY - GLAUCOMA AGENTS,%,0.005,NONSPCL,rx,0,0,0,0,0
32130,1120826442-TAGRISSO-1,M524312133731011,51672130100,2022-01-31T00:00:00.000Z,2022-02-22T00:00:00.000Z,30,35.15,14756.31,N,N,GENERIC,NONMAINT,*DERMATOLOGICALS*,*EMOLLIENTS**,DERMATOLOGY,%,12.000,NONSPCL,rx,0,0,0,0,0
32131,1121652095-TAGRISSO-1,M524683055321011,2823305,2022-03-09T00:00:00.000Z,2022-03-23T00:00:00.000Z,26,155.64,1493.13,N,N,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,-25/ML,75.000,NONSPCL,rx,0,0,0,0,0


In [ ]:
rxclms_train[rxclms_train['therapy_id']=='1009508044-TAGRISSO-1'].head(10)

,therapy_id,document_key,ndc_id,service_date,process_date,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,reversal_ind,mail_order_ind,generic_ind,maint_ind,gpi_drug_group_desc,gpi_drug_class_desc,hum_drug_class_desc,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind
0,1009508044-TAGRISSO-1,A184611654291011,169266015,2018-03-02T00:00:00.000Z,2019-12-04T00:00:00.000Z,30,919.78,2830.22,N,N,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.0,NONSPCL,rx,0,0,0,0,0
517,1009508044-TAGRISSO-1,A197509414111001,186038230,2019-12-16T00:00:00.000Z,2019-12-17T00:00:00.000Z,30,16.70,20199.68,N,N,GENERIC,MAINT,*ULCER DRUGS/ANTISPASMODICS/ANTICHOLINERGICS*,*PROTON PUMP INHIBITORS**,GASTROINTESTINAL DISEASE - GERD,MG,20.0,NONSPCL,rx,0,1,0,0,0
1306,1009508044-TAGRISSO-1,B204206221881001,169266015,2020-01-20T00:00:00.000Z,2020-01-21T00:00:00.000Z,42,1068.20,833.62,N,Y,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.0,NONSPCL,rx,0,0,0,0,0
1390,1009508044-TAGRISSO-1,B204411913981001,68180051703,2020-02-10T00:00:00.000Z,2020-02-11T00:00:00.000Z,90,7.15,19496.69,N,Y,GENERIC,MAINT,*ANTIHYPERTENSIVES*,*ACE INHIBITORS**,CARDIOLOGY - HYPERTENSION AGENTS,MG,40.0,NONSPCL,rx,0,0,0,0,0
1391,1009508044-TAGRISSO-1,B204411932891001,66993001968,2020-02-10T00:00:00.000Z,2020-02-11T00:00:00.000Z,75,113.84,19473.86,N,Y,GENERIC,MAINT,*ANTIASTHMATIC AND BRONCHODILATOR AGENTS*,*SYMPATHOMIMETICS**,RESPIRATORY - ASTHMA,MCG/ACT,108.0,NONSPCL,rx,0,0,0,0,0
1392,1009508044-TAGRISSO-1,B204412440871001,51224010750,2020-02-10T00:00:00.000Z,2020-02-11T00:00:00.000Z,90,8.60,20166.37,N,Y,GENERIC,MAINT,*ANTIDIABETICS*,*BIGUANIDES**,DIABETES,MG,750.0,NONSPCL,rx,0,0,0,0,0
1400,1009508044-TAGRISSO-1,B204444831911001,13811071910,2020-02-13T00:00:00.000Z,2020-02-14T00:00:00.000Z,7,12.96,20245.83,N,N,GENERIC,NONMAINT,*ANTI-INFECTIVE AGENTS - MISC.*,*URINARY ANTI-INFECTIVES**,ANTI-INFECTIVES,MG,100.0,NONSPCL,rx,0,0,0,0,0
1484,1009508044-TAGRISSO-1,B204656542181001,169266015,2020-03-03T00:00:00.000Z,2020-03-06T00:00:00.000Z,42,1068.20,37677.77,N,Y,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.0,NONSPCL,rx,0,0,0,0,0
1606,1009508044-TAGRISSO-1,B204936219861001,310009530,2020-04-02T00:00:00.000Z,2020-04-03T00:00:00.000Z,30,379.74,50025.62,N,N,BRANDED,MAINT,*ANTIASTHMATIC AND BRONCHODILATOR AGENTS*,*SELECTIVE PHOSPHODIESTERASE 4 (PDE4) INHIBITO...,RESPIRATORY - COPD,MCG,500.0,NONSPCL,rx,0,0,0,0,0
1787,1009508044-TAGRISSO-1,B205276860871001,51224010750,2020-05-06T00:00:00.000Z,2020-05-07T00:00:00.000Z,90,8.60,64640.50,N,Y,GENERIC,MAINT,*ANTIDIABETICS*,*BIGUANIDES**,DIABETES,MG,750.0,NONSPCL,rx,0,0,0,0,0


In [ ]:
medclms_train.isna().sum()

therapy_id                     0
medclm_key                     0
clm_unique_key                 0
primary_diag_cd                0
visit_date                     0
diag_cd2                   24130
diag_cd3                   41358
diag_cd4                   52544
diag_cd5                   61886
diag_cd6                   67371
diag_cd7                   72473
diag_cd8                   75957
diag_cd9                   78844
process_date                   0
reversal_ind               98870
pot                            0
util_cat                   43428
hedis_pot                      0
clm_type                       0
ade_diagnosis                  0
seizure_diagnosis              0
pain_diagnosis                 0
fatigue_diagnosis              0
nausea_diagnosis               0
hyperglycemia_diagnosis        0
constipation_diagnosis         0
diarrhea_diagnosis             0
dtype: int64

###document_key - unique identifier for a prescription claim document

In [ ]:
print(rxclms_train.shape[0])
print(rxclms_train['document_key'].nunique())

32133
32133


In [ ]:
#dropping this variable as it is unique id and not helpful for joining with other tables
rxclms_train.drop(['document_key'],axis=1,inplace=True)

###ndc_id  -  National Drug Code Identifier: a national/FDA identifier for a specific drug

In [ ]:
#check number of unique drugs
rxclms_train['ndc_id'].value_counts()

310135030      4169
310134930       538
3089421         418
54418425        283
59746011506     237
               ... 
555057202         1
115168774         1
16729048516       1
50228046605       1
70954048410       1
Name: ndc_id, Length: 4497, dtype: int64

In [ ]:
#dropping this variable as there are already indicator variables created for symptoms of ade
# rxclms_train.drop(['ndc_id'],axis=1,inplace=True)

According to the date given, there are 53 drugs reported to have ADE.

###service_date - Date of a prescription fill

In [ ]:
rxclms_train['service_date'].value_counts()

2022-04-01T00:00:00.000Z    115
2022-04-25T00:00:00.000Z     78
2021-10-01T00:00:00.000Z     78
2020-10-01T00:00:00.000Z     77
2022-01-24T00:00:00.000Z     70
                           ... 
2019-04-18T00:00:00.000Z      1
2019-04-12T00:00:00.000Z      1
2019-02-16T00:00:00.000Z      1
2019-02-07T00:00:00.000Z      1
2022-12-01T00:00:00.000Z      1
Name: service_date, Length: 1388, dtype: int64

In [ ]:
# convert object to date time for ease of feature creation
rxclms_train['service_date'] = pd.to_datetime(rxclms_train['service_date'])

In [ ]:
rxclms_train['service_date'].value_counts()

2022-04-01 00:00:00+00:00    115
2022-04-25 00:00:00+00:00     78
2021-10-01 00:00:00+00:00     78
2020-10-01 00:00:00+00:00     77
2022-01-24 00:00:00+00:00     70
                            ... 
2019-04-18 00:00:00+00:00      1
2019-04-12 00:00:00+00:00      1
2019-02-16 00:00:00+00:00      1
2019-02-07 00:00:00+00:00      1
2022-12-01 00:00:00+00:00      1
Name: service_date, Length: 1388, dtype: int64

This indicates date when a member started taking this prescription. Could this be before therapy start date? This could be subtracted from therapy_start_date after joining with target_data

###process_date  - Date that this claim was processed

In [ ]:
# droppping this for now as it doesn't seem to be helpful
# rxclms_train.drop(['process_date'],axis=1,inplace=True)

### pay_day_supply_cnt - The number of days supply of a drug



In [ ]:
rxclms_train['pay_day_supply_cnt'].dtype

dtype('int64')

In [ ]:
rxclms_train['pay_day_supply_cnt'].value_counts()

30     15772
90      6916
7       1407
10      1084
15       828
5        746
28       592
14       550
25       385
1        348
3        306
6        300
20       284
60       246
8        214
4        194
2        160
50       159
16       126
12       115
84       113
17       110
21        88
31        85
24        75
22        73
9         70
75        64
45        63
33        61
13        57
23        47
18        43
11        40
35        35
29        30
37        25
19        24
100       22
40        21
34        20
48        19
42        18
27        18
67        14
66        10
26        10
57         8
83         8
56         7
89         7
36         7
180        6
68         6
85         5
55         5
32         5
81         4
47         4
38         4
88         4
46         4
82         4
80         4
43         4
51         4
41         4
87         4
70         3
58         3
76         3
65         3
63         3
39         3
72         2
69         2
86         2

## Aggregated features  - rxclms

In [ ]:
# creating new features to indicate drugs taken after therapy start date
rxclms_train = rxclms_train.merge(target_train[['therapy_id','therapy_start_date']], on='therapy_id', how='left')
rxclms_train['after_therapy']= (rxclms_train['therapy_start_date'] > rxclms_train['service_date']).astype(int)
rxclms_train['ddi_after_therapy']= (rxclms_train['ddi_ind'] & rxclms_train['after_therapy'])
rxclms_train['anticoag_after_therapy']= (rxclms_train['anticoag_ind'] & rxclms_train['after_therapy'])
rxclms_train['diarrhea_after_therapy']= (rxclms_train['diarrhea_treat_ind'] & rxclms_train['after_therapy'])
rxclms_train['nausea_after_therapy']= (rxclms_train['nausea_treat_ind'] & rxclms_train['after_therapy'])
rxclms_train['seizure_after_therapy']= (rxclms_train['seizure_treat_ind'] & rxclms_train['after_therapy'])
rxclms_train.head(5)

,therapy_id,ndc_id,service_date,process_date,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,reversal_ind,mail_order_ind,generic_ind,maint_ind,gpi_drug_group_desc,gpi_drug_class_desc,hum_drug_class_desc,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind,therapy_start_date,after_therapy,ddi_after_therapy,anticoag_after_therapy,diarrhea_after_therapy,nausea_after_therapy,seizure_after_therapy
0,1009508044-TAGRISSO-1,169266015,2018-03-02 00:00:00+00:00,2019-12-04T00:00:00.000Z,30,919.78,2830.22,N,N,BRANDED,MAINT,*ANTIDIABETICS*,*INSULIN**,DIABETES,UNIT/ML,100.0,NONSPCL,rx,0,0,0,0,0,2020-01-23 00:00:00+00:00,1,0,0,0,0,0
1,1023838279-TAGRISSO-1,69097022416,2018-03-28 00:00:00+00:00,2020-03-13T00:00:00.000Z,28,5.00,1138.66,N,N,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.0,NONSPCL,rx,0,0,0,0,0,2019-10-18 00:00:00+00:00,1,0,0,0,0,0
2,1023838279-TAGRISSO-1,60505257908,2018-07-14 00:00:00+00:00,2020-03-13T00:00:00.000Z,30,7.30,2693.11,N,N,GENERIC,MAINT,*ANTIHYPERLIPIDEMICS*,*HMG COA REDUCTASE INHIBITORS**,CHOLESTEROL,MG,20.0,NONSPCL,rx,0,0,0,0,0,2019-10-18 00:00:00+00:00,1,0,0,0,0,0
3,1023838279-TAGRISSO-1,65862057290,2018-08-09 00:00:00+00:00,2020-03-17T00:00:00.000Z,30,12.20,2718.31,N,N,GENERIC,MAINT,*ANTIHYPERTENSIVES*,*ANGIOTENSIN II RECEPTOR ANTAGONISTS**,CARDIOLOGY - HYPERTENSION AGENTS,MG,160.0,NONSPCL,rx,0,0,0,0,0,2019-10-18 00:00:00+00:00,1,0,0,0,0,0
4,1071647492-TAGRISSO-1,16252060144,2019-01-01 00:00:00+00:00,2020-03-03T00:00:00.000Z,90,9.90,0.00,N,Y,GENERIC,MAINT,*ENDOCRINE AND METABOLIC AGENTS - MISC.*,*BONE DENSITY REGULATORS**,OSTEOPOROSIS,MG,70.0,NONSPCL,rx,0,0,0,0,0,2019-12-31 00:00:00+00:00,1,0,0,0,0,0


In [ ]:
rxclms_agg = rxclms_train.groupby(['therapy_id'],as_index=False).agg(
                                                      n_rx_claims = pd.NamedAgg(column = 'therapy_id', aggfunc = 'count'),
                                                      unique_ndc_id = pd.NamedAgg(column = 'ndc_id', aggfunc = lambda x: x.nunique()),
                                                      service_date_fst = pd.NamedAgg(column = 'service_date', aggfunc = 'min'),
                                                      service_date_lst = pd.NamedAgg(column = 'service_date', aggfunc = 'max'),
                                                      pay_day_supply_cnt_min = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'min'),
                                                      pay_day_supply_cnt_avg = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'mean'),
                                                      pay_day_supply_cnt_max = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'max'),
                                                      rx_cost_sum = pd.NamedAgg(column = 'rx_cost', aggfunc = 'sum'),
                                                      tot_drug_cost_accum_amt_max = pd.NamedAgg(column = 'tot_drug_cost_accum_amt', aggfunc = 'max'),
                                                      # tot_drug_cost_accum_amt_sum = pd.NamedAgg(column = 'tot_drug_cost_accum_amt', aggfunc = 'sum'),
                                                      reversal_ind_sum = pd.NamedAgg(column = 'reversal_ind', aggfunc = lambda x:(x == 'Y').sum()),
                                                      mail_order_ind_sum = pd.NamedAgg(column = 'mail_order_ind', aggfunc = lambda x:(x == 'Y').sum()),
                                                      generic_ind_sum = pd.NamedAgg(column = 'generic_ind', aggfunc = lambda x:(x == 'GENERIC').sum()),
                                                      maint_ind_sum = pd.NamedAgg(column = 'maint_ind', aggfunc = lambda x:(x == 'MAINT').sum()),
                                                      specialty_ind_sum = pd.NamedAgg(column = 'specialty_ind', aggfunc = lambda x:(x == 'SPCL').sum()),
                                                      ddi_ind_sum = pd.NamedAgg(column = 'ddi_ind', aggfunc = 'sum'),
                                                      anticoag_ind_fst = pd.NamedAgg(column = 'anticoag_ind', aggfunc = lambda x:  x.head(1)),
                                                      anticoag_ind_sum = pd.NamedAgg(column = 'anticoag_ind', aggfunc = 'sum'),
                                                      anticoag_ind_lst = pd.NamedAgg(column = 'anticoag_ind', aggfunc = lambda x: x.tail(1)),
                                                      diarrhea_treat_ind_fst = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      diarrhea_treat_ind_sum = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = 'sum'),
                                                      diarrhea_treat_ind_lst = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      nausea_treat_ind_fst = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      nausea_treat_ind_sum = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = 'sum'),
                                                      nausea_treat_ind_lst = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      seizure_treat_ind_fst = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      seizure_treat_ind_sum = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = 'sum'),
                                                      seizure_treat_ind_lst = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      #new features based on whether before/after therapy start date
                                                      ddi_after_therapy_sum = pd.NamedAgg(column = 'ddi_after_therapy', aggfunc = 'sum'),
                                                      anticoag_after_therapy_sum = pd.NamedAgg(column = 'anticoag_after_therapy', aggfunc = 'sum'),
                                                      diarrhea_after_therapy_sum = pd.NamedAgg(column = 'diarrhea_after_therapy', aggfunc = 'sum'),
                                                      nausea_after_therapy_sum = pd.NamedAgg(column = 'nausea_after_therapy', aggfunc = 'sum'),
                                                      seizure_after_therapy_sum = pd.NamedAgg(column = 'seizure_after_therapy', aggfunc = 'sum'),
                                                      )
rxclms_agg.head(5)

,therapy_id,n_rx_claims,unique_ndc_id,service_date_fst,service_date_lst,pay_day_supply_cnt_min,pay_day_supply_cnt_avg,pay_day_supply_cnt_max,rx_cost_sum,tot_drug_cost_accum_amt_max,reversal_ind_sum,mail_order_ind_sum,generic_ind_sum,maint_ind_sum,specialty_ind_sum,ddi_ind_sum,anticoag_ind_fst,anticoag_ind_sum,anticoag_ind_lst,diarrhea_treat_ind_fst,diarrhea_treat_ind_sum,diarrhea_treat_ind_lst,nausea_treat_ind_fst,nausea_treat_ind_sum,nausea_treat_ind_lst,seizure_treat_ind_fst,seizure_treat_ind_sum,seizure_treat_ind_lst,ddi_after_therapy_sum,anticoag_after_therapy_sum,diarrhea_after_therapy_sum,nausea_after_therapy_sum,seizure_after_therapy_sum
0,1000110916-TAGRISSO-1,42,24,2020-01-18 00:00:00+00:00,2020-07-09 00:00:00+00:00,1,48.833333,90,53655.47,53500.26,0,19,31,26,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
1,1000207687-TAGRISSO-1,28,23,2019-07-17 00:00:00+00:00,2019-10-16 00:00:00+00:00,1,46.071429,90,17421.35,19105.80,0,19,15,12,1,2,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,1000260673-TAGRISSO-1,32,25,2021-05-09 00:00:00+00:00,2021-10-11 00:00:00+00:00,5,64.875000,90,67749.97,52240.45,0,23,27,24,4,3,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0
3,1000389557-TAGRISSO-1,21,12,2021-09-08 00:00:00+00:00,2022-05-16 00:00:00+00:00,4,16.904762,30,115001.13,98629.06,0,0,14,2,7,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0
4,1000767323-TAGRISSO-1,13,7,2021-11-01 00:00:00+00:00,2022-03-17 00:00:00+00:00,30,76.153846,90,48917.27,48861.95,0,10,9,10,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
rxclms_agg.columns

Index(['therapy_id', 'n_rx_claims', 'unique_ndc_id', 'service_date_fst',
       'service_date_lst', 'pay_day_supply_cnt_min', 'pay_day_supply_cnt_avg',
       'pay_day_supply_cnt_max', 'rx_cost_sum', 'tot_drug_cost_accum_amt_max',
       'reversal_ind_sum', 'mail_order_ind_sum', 'generic_ind_sum',
       'maint_ind_sum', 'specialty_ind_sum', 'ddi_ind_sum', 'anticoag_ind_fst',
       'anticoag_ind_sum', 'anticoag_ind_lst', 'diarrhea_treat_ind_fst',
       'diarrhea_treat_ind_sum', 'diarrhea_treat_ind_lst',
       'nausea_treat_ind_fst', 'nausea_treat_ind_sum', 'nausea_treat_ind_lst',
       'seizure_treat_ind_fst', 'seizure_treat_ind_sum',
       'seizure_treat_ind_lst', 'ddi_after_therapy_sum',
       'anticoag_after_therapy_sum', 'diarrhea_after_therapy_sum',
       'nausea_after_therapy_sum', 'seizure_after_therapy_sum'],
      dtype='object')

In [ ]:
rxclms_agg['service_date_diff'] = (rxclms_agg['service_date_lst'] - rxclms_agg['service_date_fst']).dt.days

In [ ]:
rxclms_agg.shape

(1160, 34)

## Aggregated features  - medclms

In [ ]:
medclms_train[medclms_train['therapy_id']== '1066310426-TAGRISSO-1'].head(5)

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,diag_cd2,diag_cd3,diag_cd4,diag_cd5,diag_cd6,diag_cd7,diag_cd8,diag_cd9,process_date,reversal_ind,pot,util_cat,hedis_pot,clm_type,ade_diagnosis,seizure_diagnosis,pain_diagnosis,fatigue_diagnosis,nausea_diagnosis,hyperglycemia_diagnosis,constipation_diagnosis,diarrhea_diagnosis
0,1066310426-TAGRISSO-1,35908472910,649143000473431803,I70292,2019-09-10T00:00:00.000Z,I771,I739,R9431,Z7982,Z79899,NaN,NaN,NaN,2019-11-21T05:00:00.000Z,NaN,Outpatient,Outpatient,Other,med,0,0,0,0,0,0,0,0
1007,1066310426-TAGRISSO-1,36100106282,71832000474281803,J188,2019-12-27T00:00:00.000Z,E785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-04T05:00:00.000Z,NaN,Outpatient,Outpatient,Other,med,0,0,0,0,0,0,0,0
2370,1066310426-TAGRISSO-1,36182207389,555913000474331603,J910,2019-12-30T00:00:00.000Z,I824Z2,C3491,J449,NaN,NaN,NaN,NaN,NaN,2020-01-24T05:00:00.000Z,NaN,Physician_Office,Physician_Office,Outpatient,med,0,0,0,0,0,0,0,0
4193,1066310426-TAGRISSO-1,36160857294,903513000474451903,Z0100,2020-01-11T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-20T05:00:00.000Z,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0
5222,1066310426-TAGRISSO-1,35908472905,649143000473431803,I70292,2019-09-10T00:00:00.000Z,I771,I739,R9431,Z7982,Z79899,NaN,NaN,NaN,2019-11-21T05:00:00.000Z,NaN,Outpatient,Outpatient,Other,med,0,0,0,0,0,0,0,0


In [ ]:
medclms_train.shape

(100159, 27)

In [ ]:
medclms_train['visit_date'] = pd.to_datetime(medclms_train['visit_date'])
medclms_train['process_date'] = pd.to_datetime(medclms_train['process_date'])

medclms_train['process_time'] = (medclms_train['process_date'] - medclms_train['visit_date']).dt.days

In [ ]:
# creating new features to indicate drugs taken after therapy start date
medclms_train = medclms_train.merge(target_train[['therapy_id','therapy_start_date']], on='therapy_id', how='left')
medclms_train['after_therapy']= (medclms_train['therapy_start_date'] > medclms_train['visit_date']).astype(int)
medclms_train['ade_diagnosis_aft']= (medclms_train['ade_diagnosis'] & medclms_train['after_therapy'])
medclms_train['seizure_diagnosis_aft']= (medclms_train['seizure_diagnosis'] & medclms_train['after_therapy'])
medclms_train['pain_diagnosis_aft']= (medclms_train['pain_diagnosis'] & medclms_train['after_therapy'])
medclms_train['fatigue_diagnosis_aft']= (medclms_train['fatigue_diagnosis'] & medclms_train['after_therapy'])
medclms_train['nausea_diagnosis_aft']= (medclms_train['nausea_diagnosis'] & medclms_train['after_therapy'])
medclms_train['hyperglycemia_diagnosis_aft']= (medclms_train['hyperglycemia_diagnosis'] & medclms_train['after_therapy'])
medclms_train['constipation_diagnosis_aft']= (medclms_train['constipation_diagnosis'] & medclms_train['after_therapy'])
medclms_train['diarrhea_diagnosis_aft']= (medclms_train['diarrhea_diagnosis'] & medclms_train['after_therapy'])
medclms_train.head(5)

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,diag_cd2,diag_cd3,diag_cd4,diag_cd5,diag_cd6,diag_cd7,diag_cd8,diag_cd9,process_date,reversal_ind,pot,util_cat,hedis_pot,clm_type,ade_diagnosis,seizure_diagnosis,pain_diagnosis,fatigue_diagnosis,nausea_diagnosis,hyperglycemia_diagnosis,constipation_diagnosis,diarrhea_diagnosis,process_time,therapy_start_date,after_therapy,ade_diagnosis_aft,seizure_diagnosis_aft,pain_diagnosis_aft,fatigue_diagnosis_aft,nausea_diagnosis_aft,hyperglycemia_diagnosis_aft,constipation_diagnosis_aft,diarrhea_diagnosis_aft
0,1066310426-TAGRISSO-1,35908472910,649143000473431803,I70292,2019-09-10 00:00:00+00:00,I771,I739,R9431,Z7982,Z79899,NaN,NaN,NaN,2019-11-21 05:00:00+00:00,NaN,Outpatient,Outpatient,Other,med,0,0,0,0,0,0,0,0,72,2020-01-16 00:00:00+00:00,1,0,0,0,0,0,0,0,0
1,1094522162-TAGRISSO-1,37189420103,934834000476811903,J9601,2020-08-21 00:00:00+00:00,J910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-12 04:00:00+00:00,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0,22,2020-08-14 00:00:00+00:00,0,0,0,0,0,0,0,0,0
2,1104741185-TAGRISSO-1,37128499255,748244100476652103,E785,2020-07-14 00:00:00+00:00,I10,J324,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-28 04:00:00+00:00,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0,45,2020-11-09 00:00:00+00:00,1,0,0,0,0,0,0,0,0
3,1012749208-TAGRISSO-1,36269844999,589034000474672103,C3492,2020-02-04 00:00:00+00:00,I825Z9,T83518A,N390,J942,J910,J9811,J9819,N329,2020-02-11 05:00:00+00:00,NaN,Inpatient,IP_ACUTE,Other,med,0,0,0,0,0,0,0,0,7,2020-03-30 00:00:00+00:00,1,0,0,0,0,0,0,0,0
4,1000110916-TAGRISSO-1,36448002258,663113000474981803,C3490,2020-03-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-17 04:00:00+00:00,NaN,Physician_Office,Physician_Office,Outpatient,med,0,0,0,0,0,0,0,0,12,2020-04-14 00:00:00+00:00,1,0,0,0,0,0,0,0,0


In [ ]:
medclms_agg = medclms_train.groupby(['therapy_id'],as_index=False).agg(n_med_claims = pd.NamedAgg(column = 'therapy_id', aggfunc = 'count'),
                                                                       visit_date_fst = pd.NamedAgg(column = 'visit_date', aggfunc = lambda x: x.head(1)),
                                                                       visit_date_lst = pd.NamedAgg(column = 'visit_date', aggfunc = lambda x: x.tail(1)),
                                                                       process_date_fst = pd.NamedAgg(column = 'process_date', aggfunc = lambda x: x.head(1)),
                                                                       process_date_lst = pd.NamedAgg(column = 'process_date', aggfunc = lambda x: x.tail(1)),
                                                                       process_time_fst = pd.NamedAgg(column = 'process_time', aggfunc = lambda x: x.head(1)),
                                                                       process_time_avg = pd.NamedAgg(column = 'process_time', aggfunc = 'mean'),
                                                                       process_time_lst = pd.NamedAgg(column = 'process_time', aggfunc = lambda x: x.tail(1)),
                                                                       pot_fst = pd.NamedAgg(column = 'pot', aggfunc = lambda x: x.head(1)),
                                                                       pot_lst = pd.NamedAgg(column = 'pot', aggfunc = lambda x: x.tail(1)),
                                                                       util_cat_fst = pd.NamedAgg(column = 'util_cat', aggfunc = lambda x: x.head(1)),
                                                                       util_cat_lst = pd.NamedAgg(column = 'util_cat', aggfunc = lambda x: x.tail(1)),
                                                                       ade_diagnosis_sum = pd.NamedAgg(column = 'ade_diagnosis', aggfunc = 'sum'),
                                                                       seizure_diagnosis_sum = pd.NamedAgg(column = 'seizure_diagnosis', aggfunc = 'sum'),
                                                                       pain_diagnosis_sum = pd.NamedAgg(column = 'pain_diagnosis', aggfunc = 'sum'),
                                                                       fatigue_diagnosis_sum = pd.NamedAgg(column = 'fatigue_diagnosis', aggfunc = 'sum'),
                                                                       nausea_diagnosis_sum = pd.NamedAgg(column = 'nausea_diagnosis', aggfunc = 'sum'),
                                                                       hyperglycemia_diagnosis_sum = pd.NamedAgg(column = 'hyperglycemia_diagnosis', aggfunc = 'sum'),
                                                                       constipation_diagnosis_sum = pd.NamedAgg(column = 'constipation_diagnosis', aggfunc = 'sum'),
                                                                       diarrhea_diagnosis_sum = pd.NamedAgg(column = 'diarrhea_diagnosis', aggfunc = 'sum'),
                                                                       #after therapy features
                                                                       ade_diagnosis_aft_sum = pd.NamedAgg(column = 'ade_diagnosis_aft', aggfunc = 'sum'),
                                                                       seizure_diagnosis_aft_sum = pd.NamedAgg(column = 'seizure_diagnosis_aft', aggfunc = 'sum'),
                                                                       pain_diagnosis_aft_sum = pd.NamedAgg(column = 'pain_diagnosis_aft', aggfunc = 'sum'),
                                                                       fatigue_diagnosis_aft_sum = pd.NamedAgg(column = 'fatigue_diagnosis_aft', aggfunc = 'sum'),
                                                                       nausea_diagnosis_aft_sum = pd.NamedAgg(column = 'nausea_diagnosis_aft', aggfunc = 'sum'),
                                                                       hyperglycemia_diagnosis_aft_sum = pd.NamedAgg(column = 'hyperglycemia_diagnosis_aft', aggfunc = 'sum'),
                                                                       constipation_diagnosis_aft_sum = pd.NamedAgg(column = 'constipation_diagnosis_aft', aggfunc = 'sum'),
                                                                       diarrhea_diagnosis_aft_sum = pd.NamedAgg(column = 'diarrhea_diagnosis_aft', aggfunc = 'sum')
                                                                       )

print(medclms_agg.shape)
medclms_agg.head(5)

(536, 29)


,therapy_id,n_med_claims,visit_date_fst,visit_date_lst,process_date_fst,process_date_lst,process_time_fst,process_time_avg,process_time_lst,pot_fst,pot_lst,util_cat_fst,util_cat_lst,ade_diagnosis_sum,seizure_diagnosis_sum,pain_diagnosis_sum,fatigue_diagnosis_sum,nausea_diagnosis_sum,hyperglycemia_diagnosis_sum,constipation_diagnosis_sum,diarrhea_diagnosis_sum,ade_diagnosis_aft_sum,seizure_diagnosis_aft_sum,pain_diagnosis_aft_sum,fatigue_diagnosis_aft_sum,nausea_diagnosis_aft_sum,hyperglycemia_diagnosis_aft_sum,constipation_diagnosis_aft_sum,diarrhea_diagnosis_aft_sum
0,1000110916-TAGRISSO-1,124,2020-03-05 00:00:00+00:00,2020-03-05 00:00:00+00:00,2020-03-17 04:00:00+00:00,2020-03-17 04:00:00+00:00,12,43.419355,12,Physician_Office,Unknown,Physician_Office,NaN,8,0,0,0,7,0,1,7,7,0,0,0,7,0,0,7
1,1000207687-TAGRISSO-1,190,2018-11-17 00:00:00+00:00,2019-04-10 00:00:00+00:00,2019-10-11 04:00:00+00:00,2019-08-02 04:00:00+00:00,328,113.331579,114,Inpatient,Unknown,IP_ACUTE,NaN,5,0,0,5,0,0,0,0,5,0,0,5,0,0,0,0
2,1000260673-TAGRISSO-1,142,2021-10-22 00:00:00+00:00,2021-05-08 00:00:00+00:00,2021-10-28 04:00:00+00:00,2021-06-17 04:00:00+00:00,6,24.809859,40,Unknown,ER_Facility,NaN,ER,25,0,25,0,0,0,0,0,25,0,25,0,0,0,0,0
3,1000767323-TAGRISSO-1,65,2021-11-15 00:00:00+00:00,2021-10-01 00:00:00+00:00,2021-11-19 05:00:00+00:00,2021-10-08 04:00:00+00:00,4,12.292308,7,Outpatient,Physician_Office,Outpatient,Physician_Office,6,0,0,0,0,0,0,6,6,0,0,0,0,0,0,6
4,1000767683-TAGRISSO-1,197,2019-07-17 00:00:00+00:00,2019-07-21 00:00:00+00:00,2019-08-16 04:00:00+00:00,2019-08-15 04:00:00+00:00,30,27.822335,25,Unknown,Unknown,NaN,NaN,66,0,0,1,65,0,0,0,61,0,0,0,61,0,0,0


## Data Merge

In [ ]:
print(target_train.shape)
print(rxclms_agg.shape)
print(medclms_agg.shape)

(1232, 14)
(1160, 34)
(536, 29)


In [ ]:
df = pd.merge(target_train, rxclms_agg, on='therapy_id', how ='left')
df = pd.merge(df, medclms_agg, on='therapy_id', how ='left')
print(df.shape)

(1232, 75)


In [ ]:
df.drop(['id','member_id','therapy_end_date','drug_name','therapy_num','therapy_duration'],axis=1,inplace=True)

#creating temporal columns from service date
df['service_date_fst_start'] = (df['service_date_fst'] - df['therapy_start_date']).dt.days
df['service_date_lst_start'] = (df['service_date_lst'] - df['therapy_start_date']).dt.days

#creating temporal columns from visit date
df['visit_date_fst_start'] = (df['visit_date_fst'] - df['therapy_start_date']).dt.days
df['visit_date_lst_start'] = (df['visit_date_lst'] - df['therapy_start_date']).dt.days

#creating temporal columns from process date
df['process_date_fst_start'] = (df['process_date_fst'] - df['therapy_start_date']).dt.days
df['process_date_lst_start'] = (df['process_date_lst'] - df['therapy_start_date']).dt.days

# process - visit


df.drop(['service_date_fst','service_date_lst','therapy_start_date'],axis=1,inplace=True)
df.drop([ 'visit_date_fst', 'visit_date_lst', 'process_date_fst', 'process_date_lst'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['therapy_id', 'tgt_ade_dc_ind', 'race_cd', 'est_age', 'sex_cd',
       'cms_disabled_ind', 'cms_low_income_ind', 'n_rx_claims',
       'unique_ndc_id', 'pay_day_supply_cnt_min', 'pay_day_supply_cnt_avg',
       'pay_day_supply_cnt_max', 'rx_cost_sum', 'tot_drug_cost_accum_amt_max',
       'reversal_ind_sum', 'mail_order_ind_sum', 'generic_ind_sum',
       'maint_ind_sum', 'specialty_ind_sum', 'ddi_ind_sum', 'anticoag_ind_fst',
       'anticoag_ind_sum', 'anticoag_ind_lst', 'diarrhea_treat_ind_fst',
       'diarrhea_treat_ind_sum', 'diarrhea_treat_ind_lst',
       'nausea_treat_ind_fst', 'nausea_treat_ind_sum', 'nausea_treat_ind_lst',
       'seizure_treat_ind_fst', 'seizure_treat_ind_sum',
       'seizure_treat_ind_lst', 'ddi_after_therapy_sum',
       'anticoag_after_therapy_sum', 'diarrhea_after_therapy_sum',
       'nausea_after_therapy_sum', 'seizure_after_therapy_sum',
       'service_date_diff', 'n_med_claims', 'process_time_fst',
       'process_time_avg', 'process_time_l

In [ ]:
# numeric columns
numeric_cols = [col for col in df.columns if df[col].dtype !='O']
print(numeric_cols)

['tgt_ade_dc_ind', 'race_cd', 'est_age', 'cms_disabled_ind', 'cms_low_income_ind', 'n_rx_claims', 'unique_ndc_id', 'pay_day_supply_cnt_min', 'pay_day_supply_cnt_avg', 'pay_day_supply_cnt_max', 'rx_cost_sum', 'tot_drug_cost_accum_amt_max', 'reversal_ind_sum', 'mail_order_ind_sum', 'generic_ind_sum', 'maint_ind_sum', 'specialty_ind_sum', 'ddi_ind_sum', 'anticoag_ind_fst', 'anticoag_ind_sum', 'anticoag_ind_lst', 'diarrhea_treat_ind_fst', 'diarrhea_treat_ind_sum', 'diarrhea_treat_ind_lst', 'nausea_treat_ind_fst', 'nausea_treat_ind_sum', 'nausea_treat_ind_lst', 'seizure_treat_ind_fst', 'seizure_treat_ind_sum', 'seizure_treat_ind_lst', 'ddi_after_therapy_sum', 'anticoag_after_therapy_sum', 'diarrhea_after_therapy_sum', 'nausea_after_therapy_sum', 'seizure_after_therapy_sum', 'service_date_diff', 'n_med_claims', 'process_time_fst', 'process_time_avg', 'process_time_lst', 'ade_diagnosis_sum', 'seizure_diagnosis_sum', 'pain_diagnosis_sum', 'fatigue_diagnosis_sum', 'nausea_diagnosis_sum', 'hyper

In [ ]:
df[numeric_cols].describe()

,tgt_ade_dc_ind,race_cd,est_age,cms_disabled_ind,cms_low_income_ind,n_rx_claims,unique_ndc_id,pay_day_supply_cnt_min,pay_day_supply_cnt_avg,pay_day_supply_cnt_max,rx_cost_sum,tot_drug_cost_accum_amt_max,reversal_ind_sum,mail_order_ind_sum,generic_ind_sum,maint_ind_sum,specialty_ind_sum,ddi_ind_sum,anticoag_ind_fst,anticoag_ind_sum,anticoag_ind_lst,diarrhea_treat_ind_fst,diarrhea_treat_ind_sum,diarrhea_treat_ind_lst,nausea_treat_ind_fst,nausea_treat_ind_sum,nausea_treat_ind_lst,seizure_treat_ind_fst,seizure_treat_ind_sum,seizure_treat_ind_lst,ddi_after_therapy_sum,anticoag_after_therapy_sum,diarrhea_after_therapy_sum,nausea_after_therapy_sum,seizure_after_therapy_sum,service_date_diff,n_med_claims,process_time_fst,process_time_avg,process_time_lst,ade_diagnosis_sum,seizure_diagnosis_sum,pain_diagnosis_sum,fatigue_diagnosis_sum,nausea_diagnosis_sum,hyperglycemia_diagnosis_sum,constipation_diagnosis_sum,diarrhea_diagnosis_sum,ade_diagnosis_aft_sum,seizure_diagnosis_aft_sum,pain_diagnosis_aft_sum,fatigue_diagnosis_aft_sum,nausea_diagnosis_aft_sum,hyperglycemia_diagnosis_aft_sum,constipation_diagnosis_aft_sum,diarrhea_diagnosis_aft_sum,service_date_fst_start,service_date_lst_start,visit_date_fst_start,visit_date_lst_start,process_date_fst_start,process_date_lst_start
count,1232.000000,1164.000000,1149.000000,1149.000000,1149.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.0,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,1160.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,1160.000000,1160.000000,536.000000,536.000000,536.000000,536.000000
mean,0.094968,1.673540,73.771976,0.147084,0.388164,27.700862,14.320690,8.925000,39.351360,77.550862,68253.540707,61828.524224,0.0,8.761207,18.971552,14.615517,4.428448,0.354310,0.056034,1.434483,0.048276,0.005172,0.183621,0.009483,0.031897,0.856897,0.025000,0.005172,0.191379,0.010345,0.145690,0.480172,0.044828,0.359483,0.075862,161.475862,186.863806,36.080224,33.308788,27.800373,12.776119,0.621269,0.117537,5.110075,3.539179,0.457090,2.656716,2.054104,7.753731,0.373134,0.104478,2.893657,2.328358,0.285448,1.772388,0.720149,-59.882759,101.593103,-13.154851,-0.985075,22.925373,26.815299
std,0.293289,1.315934,8.517059,0.354344,0.487544,27.399218,10.168923,10.791557,13.602803,25.428561,44685.957125,44200.124043,0.0,11.290228,21.472981,21.555405,3.043836,1.528949,0.230087,2.727013,0.214441,0.071764,0.747668,0.096958,0.175800,1.957760,0.156192,0.071764,0.906079,0.101226,0.804634,1.105147,0.245176,0.920993,0.388937,94.529366,145.055304,67.995898,29.798418,48.337204,24.563370,4.069058,1.263133,13.245666,13.463358,3.959376,11.620545,9.699160,19.004549,3.022462,1.242660,9.748131,11.485400,2.510288,10.008803,5.236469,56.831586,70.050628,90.234172,80.266075,63.380899,65.662643
min,0.000000,0.000000,38.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-692.000000,-83.000000,-843.000000,-622.000000,-88.000000,-90.000000
25%,0.000000,1.000000,68.000000,0.000000,0.000000,11.000000,7.000000,2.000000,29.719066,90.000000,19498.235000,17859.602500,0.0,1.000000,7.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

## Data Store

In [ ]:
df.to_csv('/content/drive/Shareddrives/Humana_mays_Competetion/Data/Training_1005.csv',index=False)

## Prep Holdout Data

In [ ]:
target_holdout = target_holdout[['id','therapy_id', 'race_cd', 'est_age', 'sex_cd','cms_disabled_ind', 'cms_low_income_ind','therapy_start_date']]

rxclms_holdout['service_date'] = pd.to_datetime(rxclms_holdout['service_date'])

medclms_holdout['visit_date'] = pd.to_datetime(medclms_holdout['visit_date'])
medclms_holdout['process_date'] = pd.to_datetime(medclms_holdout['process_date'])
medclms_holdout['process_time'] = (medclms_holdout['process_date'] - medclms_holdout['visit_date']).dt.days

In [ ]:
# creating new features to indicate drugs taken after therapy start date
rxclms_holdout = rxclms_holdout.merge(target_holdout[['therapy_id','therapy_start_date']], on='therapy_id', how='left')
rxclms_holdout['after_therapy']= (rxclms_holdout['therapy_start_date'] > rxclms_holdout['service_date']).astype(int)
rxclms_holdout['anticoag_after_therapy']= (rxclms_holdout['anticoag_ind'] & rxclms_holdout['after_therapy'])
rxclms_holdout['ddi_after_therapy']= (rxclms_holdout['ddi_ind'] & rxclms_holdout['after_therapy'])
rxclms_holdout['diarrhea_after_therapy']= (rxclms_holdout['diarrhea_treat_ind'] & rxclms_holdout['after_therapy'])
rxclms_holdout['nausea_after_therapy']= (rxclms_holdout['nausea_treat_ind'] & rxclms_holdout['after_therapy'])
rxclms_holdout['seizure_after_therapy']= (rxclms_holdout['seizure_treat_ind'] & rxclms_holdout['after_therapy'])
rxclms_holdout.head(5)

,therapy_id,document_key,ndc_id,service_date,process_date,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,reversal_ind,mail_order_ind,generic_ind,maint_ind,gpi_drug_group_desc,gpi_drug_class_desc,hum_drug_class_desc,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind,therapy_start_date,after_therapy,anticoag_after_therapy,ddi_after_therapy,diarrhea_after_therapy,nausea_after_therapy,seizure_after_therapy
0,1016317684-TAGRISSO-1,A195005006341001,228253950,2019-04-10 00:00:00+00:00,2019-04-11T00:00:00.000+0000,30,13.70,4725.09,N,N,GENERIC,MAINT,*ANTIPARKINSON AND RELATED THERAPY AGENTS*,*ANTIPARKINSON DOPAMINERGICS**,NEUROLOGY - PARKINSON AGENTS,MG-100MG,25.0,NONSPCL,rx,0,0,0,0,0,2019-07-08T00:00:00.000+0000,1,0,0,0,0,0
1,1084085456-TAGRISSO-1,A195044963981001,29300024210,2019-04-14 00:00:00+00:00,2019-04-15T00:00:00.000+0000,30,1.33,1228.73,N,N,GENERIC,MAINT,*CALCIUM CHANNEL BLOCKERS*,*CALCIUM CHANNEL BLOCKERS**,CARDIOLOGY - HYPERTENSION AGENTS,MG,5.0,NONSPCL,rx,0,0,0,0,0,2019-07-01T00:00:00.000+0000,1,0,0,0,0,0
2,1016343065-TAGRISSO-1,A195078392391001,65862019899,2019-04-17 00:00:00+00:00,2019-04-18T00:00:00.000+0000,90,5.36,6.16,N,Y,GENERIC,MAINT,*ANTICONVULSANTS*,*ANTICONVULSANTS - MISC.**,ANTICONVULSANTS,MG,100.0,NONSPCL,rx,0,0,0,0,0,2019-07-03T00:00:00.000+0000,1,0,0,0,0,0
3,1096559829-TAGRISSO-1,A195157798591001,50242006401,2019-04-25 00:00:00+00:00,2019-04-26T00:00:00.000+0000,30,9043.00,27129.00,N,N,BRANDED,MAINT,*ANTINEOPLASTICS AND ADJUNCTIVE THERAPIES*,*ANTINEOPLASTIC - EGFR INHIBITORS**,CHEMOTHERAPY,MG,150.0,SPCL,rx,0,0,0,0,0,2019-07-19T00:00:00.000+0000,1,0,0,0,0,0
4,1016317684-TAGRISSO-1,A195223757081001,60505082901,2019-05-02 00:00:00+00:00,2019-05-03T00:00:00.000+0000,30,6.10,4738.79,N,N,GENERIC,MAINT,*NASAL AGENTS - SYSTEMIC AND TOPICAL*,*NASAL STEROIDS**,COUGH/ COLD/ ALLERGY,MCG/ACT,50.0,NONSPCL,rx,0,0,0,0,0,2019-07-08T00:00:00.000+0000,1,0,0,0,0,0


In [ ]:
rxclms_agg_holdout = rxclms_holdout.groupby(['therapy_id'],as_index=False).agg(
                                                      n_rx_claims = pd.NamedAgg(column = 'therapy_id', aggfunc = 'count'),
                                                      unique_ndc_id = pd.NamedAgg(column = 'ndc_id', aggfunc = lambda x: x.nunique()),
                                                      service_date_fst = pd.NamedAgg(column = 'service_date', aggfunc = 'min'),
                                                      service_date_lst = pd.NamedAgg(column = 'service_date', aggfunc = 'max'),
                                                      # service_date_diff = pd.NamedAgg(column = 'service_date', aggfunc = lambda x: (x.tail(1)-x.head(1)).dt.days),
                                                      pay_day_supply_cnt_min = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'min'),
                                                      pay_day_supply_cnt_avg = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'mean'),
                                                      pay_day_supply_cnt_max = pd.NamedAgg(column = 'pay_day_supply_cnt', aggfunc = 'max'),
                                                      rx_cost_sum = pd.NamedAgg(column = 'rx_cost', aggfunc = 'sum'),
                                                      tot_drug_cost_accum_amt_max = pd.NamedAgg(column = 'tot_drug_cost_accum_amt', aggfunc = 'max'),
                                                      # tot_drug_cost_accum_amt_sum = pd.NamedAgg(column = 'tot_drug_cost_accum_amt', aggfunc = 'sum'),
                                                      reversal_ind_sum = pd.NamedAgg(column = 'reversal_ind', aggfunc = lambda x:(x == 'Y').sum()),
                                                      mail_order_ind_sum = pd.NamedAgg(column = 'mail_order_ind', aggfunc = lambda x:(x == 'Y').sum()),
                                                      generic_ind_sum = pd.NamedAgg(column = 'generic_ind', aggfunc = lambda x:(x == 'GENERIC').sum()),
                                                      maint_ind_sum = pd.NamedAgg(column = 'maint_ind', aggfunc = lambda x:(x == 'MAINT').sum()),
                                                      specialty_ind_sum = pd.NamedAgg(column = 'specialty_ind', aggfunc = lambda x:(x == 'SPCL').sum()),
                                                      ddi_ind_sum = pd.NamedAgg(column = 'ddi_ind', aggfunc = 'sum'),
                                                      anticoag_ind_fst = pd.NamedAgg(column = 'anticoag_ind', aggfunc = lambda x:  x.head(1)),
                                                      anticoag_ind_sum = pd.NamedAgg(column = 'anticoag_ind', aggfunc = 'sum'),
                                                      anticoag_ind_lst = pd.NamedAgg(column = 'anticoag_ind', aggfunc = lambda x: x.tail(1)),
                                                      diarrhea_treat_ind_fst = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      diarrhea_treat_ind_sum = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = 'sum'),
                                                      diarrhea_treat_ind_lst = pd.NamedAgg(column = 'diarrhea_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      nausea_treat_ind_fst = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      nausea_treat_ind_sum = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = 'sum'),
                                                      nausea_treat_ind_lst = pd.NamedAgg(column = 'nausea_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      seizure_treat_ind_fst = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = lambda x: x.head(1)),
                                                      seizure_treat_ind_sum = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = 'sum'),
                                                      seizure_treat_ind_lst = pd.NamedAgg(column = 'seizure_treat_ind', aggfunc = lambda x: x.tail(1)),
                                                      #new features based on whether before/after therapy start date
                                                      ddi_after_therapy_sum = pd.NamedAgg(column = 'ddi_after_therapy', aggfunc = 'sum'),
                                                      anticoag_after_therapy_sum = pd.NamedAgg(column = 'anticoag_after_therapy', aggfunc = 'sum'),
                                                      diarrhea_after_therapy_sum = pd.NamedAgg(column = 'diarrhea_after_therapy', aggfunc = 'sum'),
                                                      nausea_after_therapy_sum = pd.NamedAgg(column = 'nausea_after_therapy', aggfunc = 'sum'),
                                                      seizure_after_therapy_sum = pd.NamedAgg(column = 'seizure_after_therapy', aggfunc = 'sum'),
                                                      )
rxclms_agg_holdout.head(5)

,therapy_id,n_rx_claims,unique_ndc_id,service_date_fst,service_date_lst,pay_day_supply_cnt_min,pay_day_supply_cnt_avg,pay_day_supply_cnt_max,rx_cost_sum,tot_drug_cost_accum_amt_max,reversal_ind_sum,mail_order_ind_sum,generic_ind_sum,maint_ind_sum,specialty_ind_sum,ddi_ind_sum,anticoag_ind_fst,anticoag_ind_sum,anticoag_ind_lst,diarrhea_treat_ind_fst,diarrhea_treat_ind_sum,diarrhea_treat_ind_lst,nausea_treat_ind_fst,nausea_treat_ind_sum,nausea_treat_ind_lst,seizure_treat_ind_fst,seizure_treat_ind_sum,seizure_treat_ind_lst,ddi_after_therapy_sum,anticoag_after_therapy_sum,diarrhea_after_therapy_sum,nausea_after_therapy_sum,seizure_after_therapy_sum
0,1000261259-TAGRISSO-1,9,8,2022-01-13 00:00:00+00:00,2022-02-16 00:00:00+00:00,30,63.333333,90,19586.28,3313.22,0,6,8,7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001691228-TAGRISSO-1,1,1,2020-12-01 00:00:00+00:00,2020-12-01 00:00:00+00:00,1,1.000000,1,2.11,117722.32,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1002060555-TAGRISSO-1,7,7,2019-05-21 00:00:00+00:00,2019-08-05 00:00:00+00:00,1,14.428571,30,16715.77,3389.43,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1003935675-TAGRISSO-1,14,11,2021-08-15 00:00:00+00:00,2021-11-01 00:00:00+00:00,5,75.357143,90,16254.54,16767.83,0,11,13,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1004837018-TAGRISSO-1,38,31,2019-07-09 00:00:00+00:00,2019-10-11 00:00:00+00:00,1,25.500000,90,6955.27,18198.93,0,4,32,15,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,0


In [ ]:
rxclms_agg_holdout['service_date_diff'] = (rxclms_agg_holdout['service_date_lst'] - rxclms_agg_holdout['service_date_fst']).dt.days

In [ ]:
# creating new features to indicate drugs taken after therapy start date
medclms_holdout = medclms_holdout.merge(target_holdout[['therapy_id','therapy_start_date']], on='therapy_id', how='left')
medclms_holdout['after_therapy']= (medclms_holdout['therapy_start_date'] > medclms_holdout['visit_date']).astype(int)
medclms_holdout['ade_diagnosis_aft']= (medclms_holdout['ade_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['seizure_diagnosis_aft']= (medclms_holdout['seizure_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['pain_diagnosis_aft']= (medclms_holdout['pain_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['fatigue_diagnosis_aft']= (medclms_holdout['fatigue_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['nausea_diagnosis_aft']= (medclms_holdout['nausea_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['hyperglycemia_diagnosis_aft']= (medclms_holdout['hyperglycemia_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['constipation_diagnosis_aft']= (medclms_holdout['constipation_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout['diarrhea_diagnosis_aft']= (medclms_holdout['diarrhea_diagnosis'] & medclms_holdout['after_therapy'])
medclms_holdout.head(5)

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,diag_cd2,diag_cd3,diag_cd4,diag_cd5,diag_cd6,diag_cd7,diag_cd8,diag_cd9,process_date,reversal_ind,pot,util_cat,hedis_pot,clm_type,ade_diagnosis,seizure_diagnosis,pain_diagnosis,fatigue_diagnosis,nausea_diagnosis,hyperglycemia_diagnosis,constipation_diagnosis,diarrhea_diagnosis,process_time,therapy_start_date,after_therapy,ade_diagnosis_aft,seizure_diagnosis_aft,pain_diagnosis_aft,fatigue_diagnosis_aft,nausea_diagnosis_aft,hyperglycemia_diagnosis_aft,constipation_diagnosis_aft,diarrhea_diagnosis_aft
0,1057995894-TAGRISSO-1,37728229632,117823000477912403,E43,2020-12-17 00:00:00+00:00,J189,C3492,C7931,I4820,F5000,J9811,J90,N179,2020-12-31 05:00:00+00:00,NaN,Inpatient,IP_ACUTE,Inpatient,med,0,0,0,0,0,0,0,0,14,2020-12-29T00:00:00.000+0000,1,0,0,0,0,0,0,0,0
1,1101829515-TAGRISSO-1,38488780655,146472100479322603,R779,2021-04-29 00:00:00+00:00,D649,G629,R519,NaN,NaN,NaN,NaN,NaN,2021-05-27 04:00:00+00:00,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0,28,2021-07-29T00:00:00.000+0000,1,0,0,0,0,0,0,0,0
2,1062416207-TAGRISSO-1,39449274289,75203100481052003,I10,2021-11-01 00:00:00+00:00,E559,G8929,E785,Z7689,Z23,Z1159,K219,J449,2021-11-17 05:00:00+00:00,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0,16,2021-12-28T00:00:00.000+0000,1,0,0,0,0,0,0,0,0
3,1069511399-TAGRISSO-1,39711591440,450719000481542103,C7951,2021-12-10 00:00:00+00:00,C801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 05:00:00+00:00,NaN,Unknown,NaN,Other,med,0,0,0,0,0,0,0,0,21,2022-01-06T00:00:00.000+0000,1,0,0,0,0,0,0,0,0
4,1072992888-TAGRISSO-1,39337288280,236568000480921603,R5383,2021-10-07 00:00:00+00:00,Z0000,Z79899,Z13220,R609,NaN,NaN,NaN,NaN,2021-10-30 04:00:00+00:00,NaN,Outpatient,Outpatient,Other,med,1,0,0,1,0,0,0,0,23,2021-12-01T00:00:00.000+0000,1,1,0,0,1,0,0,0,0


In [ ]:
medclms_agg_holdout = medclms_holdout.groupby(['therapy_id'],as_index=False).agg(n_med_claims = pd.NamedAgg(column = 'therapy_id', aggfunc = 'count'),
                                                                       visit_date_fst = pd.NamedAgg(column = 'visit_date', aggfunc = lambda x: x.head(1)),
                                                                       visit_date_lst = pd.NamedAgg(column = 'visit_date', aggfunc = lambda x: x.tail(1)),
                                                                       process_date_fst = pd.NamedAgg(column = 'process_date', aggfunc = lambda x: x.head(1)),
                                                                       process_date_lst = pd.NamedAgg(column = 'process_date', aggfunc = lambda x: x.tail(1)),
                                                                       process_time_fst = pd.NamedAgg(column = 'process_time', aggfunc = lambda x: x.head(1)),
                                                                       process_time_avg = pd.NamedAgg(column = 'process_time', aggfunc = 'mean'),
                                                                       process_time_lst = pd.NamedAgg(column = 'process_time', aggfunc = lambda x: x.tail(1)),
                                                                       pot_fst = pd.NamedAgg(column = 'pot', aggfunc = lambda x: x.head(1)),
                                                                       pot_lst = pd.NamedAgg(column = 'pot', aggfunc = lambda x: x.tail(1)),
                                                                       util_cat_fst = pd.NamedAgg(column = 'util_cat', aggfunc = lambda x: x.head(1)),
                                                                       util_cat_lst = pd.NamedAgg(column = 'util_cat', aggfunc = lambda x: x.tail(1)),

                                                                       ade_diagnosis_sum = pd.NamedAgg(column = 'ade_diagnosis', aggfunc = 'sum'),
                                                                       seizure_diagnosis_sum = pd.NamedAgg(column = 'seizure_diagnosis', aggfunc = 'sum'),
                                                                       pain_diagnosis_sum = pd.NamedAgg(column = 'pain_diagnosis', aggfunc = 'sum'),
                                                                       fatigue_diagnosis_sum = pd.NamedAgg(column = 'fatigue_diagnosis', aggfunc = 'sum'),
                                                                       nausea_diagnosis_sum = pd.NamedAgg(column = 'nausea_diagnosis', aggfunc = 'sum'),
                                                                       hyperglycemia_diagnosis_sum = pd.NamedAgg(column = 'hyperglycemia_diagnosis', aggfunc = 'sum'),
                                                                       constipation_diagnosis_sum = pd.NamedAgg(column = 'constipation_diagnosis', aggfunc = 'sum'),
                                                                       diarrhea_diagnosis_sum = pd.NamedAgg(column = 'diarrhea_diagnosis', aggfunc = 'sum'),

                                                                       #after therapy features
                                                                       ade_diagnosis_aft_sum = pd.NamedAgg(column = 'ade_diagnosis_aft', aggfunc = 'sum'),
                                                                       seizure_diagnosis_aft_sum = pd.NamedAgg(column = 'seizure_diagnosis_aft', aggfunc = 'sum'),
                                                                       pain_diagnosis_aft_sum = pd.NamedAgg(column = 'pain_diagnosis_aft', aggfunc = 'sum'),
                                                                       fatigue_diagnosis_aft_sum = pd.NamedAgg(column = 'fatigue_diagnosis_aft', aggfunc = 'sum'),
                                                                       nausea_diagnosis_aft_sum = pd.NamedAgg(column = 'nausea_diagnosis_aft', aggfunc = 'sum'),
                                                                       hyperglycemia_diagnosis_aft_sum = pd.NamedAgg(column = 'hyperglycemia_diagnosis_aft', aggfunc = 'sum'),
                                                                       constipation_diagnosis_aft_sum = pd.NamedAgg(column = 'constipation_diagnosis_aft', aggfunc = 'sum'),
                                                                       diarrhea_diagnosis_aft_sum = pd.NamedAgg(column = 'diarrhea_diagnosis_aft', aggfunc = 'sum')

                                                                       )

print(medclms_agg_holdout.shape)
medclms_agg_holdout.head(5)

(185, 29)


,therapy_id,n_med_claims,visit_date_fst,visit_date_lst,process_date_fst,process_date_lst,process_time_fst,process_time_avg,process_time_lst,pot_fst,pot_lst,util_cat_fst,util_cat_lst,ade_diagnosis_sum,seizure_diagnosis_sum,pain_diagnosis_sum,fatigue_diagnosis_sum,nausea_diagnosis_sum,hyperglycemia_diagnosis_sum,constipation_diagnosis_sum,diarrhea_diagnosis_sum,ade_diagnosis_aft_sum,seizure_diagnosis_aft_sum,pain_diagnosis_aft_sum,fatigue_diagnosis_aft_sum,nausea_diagnosis_aft_sum,hyperglycemia_diagnosis_aft_sum,constipation_diagnosis_aft_sum,diarrhea_diagnosis_aft_sum
0,1000261259-TAGRISSO-1,118,2021-12-01 00:00:00+00:00,2022-01-26 00:00:00+00:00,2021-12-09 05:00:00+00:00,2022-02-03 05:00:00+00:00,8,16.822034,8,Unknown,Unknown,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001691228-TAGRISSO-1,26,2020-12-03 00:00:00+00:00,2020-12-03 00:00:00+00:00,2020-12-26 05:00:00+00:00,2020-12-10 05:00:00+00:00,23,12.153846,7,Physician_Office,Unknown,Physician_Office,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1002060555-TAGRISSO-1,90,2019-07-11 00:00:00+00:00,2019-06-17 00:00:00+00:00,2019-07-25 04:00:00+00:00,2019-07-02 04:00:00+00:00,14,22.811111,15,Unknown,Unknown,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1003935675-TAGRISSO-1,53,2021-09-17 00:00:00+00:00,2021-09-17 00:00:00+00:00,2021-09-23 04:00:00+00:00,2021-09-23 04:00:00+00:00,6,13.867925,6,Unknown,Unknown,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1004837018-TAGRISSO-1,106,2019-10-03 00:00:00+00:00,2019-08-27 00:00:00+00:00,2019-10-10 04:00:00+00:00,2019-09-12 04:00:00+00:00,7,14.056604,16,Outpatient,Outpatient,Outpatient,Outpatient,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_holdout = pd.merge(target_holdout, rxclms_agg_holdout, on='therapy_id', how ='left')
df_holdout = pd.merge(df_holdout, medclms_agg_holdout, on='therapy_id', how ='left')

df_holdout['therapy_start_date'] = pd.to_datetime(df_holdout['therapy_start_date'])
df_holdout['service_date_fst'] = pd.to_datetime(df_holdout['service_date_fst'])
df_holdout['service_date_lst'] = pd.to_datetime(df_holdout['service_date_lst'])


#creating temporal columns from service date
df_holdout['service_date_fst_start'] = (df_holdout['service_date_fst'] - df_holdout['therapy_start_date']).dt.days
df_holdout['service_date_lst_start'] = (df_holdout['service_date_lst'] - df_holdout['therapy_start_date']).dt.days

#creating temporal columns from visit date
df_holdout['visit_date_fst_start'] = (df_holdout['visit_date_fst'] - df_holdout['therapy_start_date']).dt.days
df_holdout['visit_date_lst_start'] = (df_holdout['visit_date_lst'] - df_holdout['therapy_start_date']).dt.days

#creating temporal columns from process date
df_holdout['process_date_fst_start'] = (df_holdout['process_date_fst'] - df_holdout['therapy_start_date']).dt.days
df_holdout['process_date_lst_start'] = (df_holdout['process_date_lst'] - df_holdout['therapy_start_date']).dt.days

df_holdout.drop(['service_date_fst','service_date_lst','therapy_start_date'],axis=1,inplace=True)
df_holdout.drop([ 'visit_date_fst', 'visit_date_lst', 'process_date_fst', 'process_date_lst'], axis=1, inplace=True)


print(df_holdout.shape)

(420, 68)


In [ ]:
df_holdout.columns

Index(['id', 'therapy_id', 'race_cd', 'est_age', 'sex_cd', 'cms_disabled_ind',
       'cms_low_income_ind', 'n_rx_claims', 'unique_ndc_id',
       'pay_day_supply_cnt_min', 'pay_day_supply_cnt_avg',
       'pay_day_supply_cnt_max', 'rx_cost_sum', 'tot_drug_cost_accum_amt_max',
       'reversal_ind_sum', 'mail_order_ind_sum', 'generic_ind_sum',
       'maint_ind_sum', 'specialty_ind_sum', 'ddi_ind_sum', 'anticoag_ind_fst',
       'anticoag_ind_sum', 'anticoag_ind_lst', 'diarrhea_treat_ind_fst',
       'diarrhea_treat_ind_sum', 'diarrhea_treat_ind_lst',
       'nausea_treat_ind_fst', 'nausea_treat_ind_sum', 'nausea_treat_ind_lst',
       'seizure_treat_ind_fst', 'seizure_treat_ind_sum',
       'seizure_treat_ind_lst', 'ddi_after_therapy_sum',
       'anticoag_after_therapy_sum', 'diarrhea_after_therapy_sum',
       'nausea_after_therapy_sum', 'seizure_after_therapy_sum',
       'service_date_diff', 'n_med_claims', 'process_time_fst',
       'process_time_avg', 'process_time_lst', 'pot_fs

In [ ]:
df_holdout.to_csv('/content/drive/Shareddrives/Humana_mays_Competetion/Data/Holdout_1005.csv',index=False)